We start by loading the population functions from vectorized_funs.py and loading necessary packages 

In [1]:
%run vectorized_funs.py 
import matplotlib
# matplotlib.use('Agg')
import matplotlib.pyplot as plt
import pandas as pd

# ABC stuff
from pyabc.visualization import plot_kde_matrix
from pyabc.visualization import plot_kde_1d

import os
import tempfile

import scipy.stats as st
import scipy as scp


from pyabc import (ABCSMC, RV, Distribution,
                   PercentileDistanceFunction)

import pyabc


## Loading data to test 
Here we load the data from different sources, including the games played. 

In [9]:
with open('data_dicts.pkl', 'rb') as input: 
    load_dict = pickle.load(input)
    init_strats_dict = load_dict["init_strats_dict"]
    init_p1_dict = load_dict["init_p1_dict"]
    init_p2_dict = load_dict["init_p2_dict"]
    payoffs_dict = load_dict["payoffs_dict"]
    role_dict = load_dict["role_dict"]
    id_dict = load_dict["id_dict"]
    actual_plays_dict = load_dict["actual_plays_dict"]


for gid in range(1,4):
    payoffs_dict[gid][0] = payoffs_dict[gid][gid-1][0].astype(float)
    payoffs_dict[gid][1] = payoffs_dict[gid][gid-1][1].astype(float)
    

In [7]:
payoffs_dict

{1: 0    [[[10, 0, 11], [12, 10, 5], [0, 12, 10]], [[10...
 Name: payoffs, dtype: object,
 2: 1    [[[2, 2, 2, 2, 2], [1, 4, 4, 4, 4], [1, 3, 10,...
 Name: payoffs, dtype: object,
 3: 2    [[[2, 4, 0], [4, 12, 0], [0, 14, 2], [6, 6, 6]...
 Name: payoffs, dtype: object}

## Setting parameters for the simulations

In [ ]:
data_dict = pseudo_data_correct
gids = [2,3,4,5,6]
n_runs = len(data_dict["LK_1"][2])
bw = 0.05
p1_size = 17
p2_size = 17 
rounds = 29
games = payoffs_dict
n_particles = 1000
max_pops = 15
min_accept_rate = 0.001

# Some custom functions 
- Functions for transforming the simulated data into long vectors that can be saved in the database and back again so that the distance can be calculated. 
- Custom distance function that generates an approximate pdf from the simulated data using kde and then calculates likelihood of the actual observations. The distance is measured as 1 - the quotient between the maximum (getting exactly the same trajectories) and the actual.  
- The likelihood estimation function which takes two sets of observations, generates a kde from the first set of observations and returns the total logpdf of the second set. 


In [10]:
def calc_likelihood(simulated_flat, hists_flat):
    simulated = unflatten_data(simulated_flat, shape)
    hists = unflatten_data(hists_flat, shape) 
    tot_score = 0 
    for sim, hist in zip(simulated, hists):
        kde = KernelDensity(bandwidth=bw)
        kde.fit(sim)
        tot_score += kde.score(hist)
    return tot_score

def get_gid_data(gids):
    return {pseudo:{gid:pseudo_data_correct[pseudo][gid] for gid in gids} for pseudo in pseudo_data_correct}

def flatten_data(hists):
    shape_vec = []
    hists_vec = np.array([])
    for hist in hists:
        shape_vec.append(hist.shape)
        hists_vec = np.append(hists_vec, hist.flatten())
    return(hists_vec,shape_vec)

def unflatten_data(hists_vec, shape_vec):
    start = 0
    end = 0 
    hists = []
    for shape in shape_vec:
        end += shape[0]*shape[1]
        hists.append(np.reshape(hists_vec[start:end],shape))
        start += shape[0]*shape[1] 
    return hists

def plot_models(history):
    for i in history.alive_models(history.max_t):
        df, w = history.get_distribution(m=i)
        df_copy = df.copy()
        model = model_names[i]
        for param in αβ_params[model]:
            a, b = αβ_lims[model][param]
            df[param] = df_copy.apply(lambda x: st.beta.mean(x[param], x[param+"_sd"])*(b-a) + a, axis=1)
            df[param+"_sd"] = df_copy.apply(lambda x: st.beta.std(x[param], x[param+"_sd"])*(b-a), axis=1)
        plot = plot_kde_matrix(df, w, limits=αβ_param_spaces[model_names[i]])
        plot.savefig("fig/SSE/abc-beta" + str(n_particles) + "-" + str(max_pops)+ "-bw" + str(bw) + "-"  + pseduo_pop  + "-" + model_names[i] + ".png")
    plt.close("all")
    
def αβ_from_μσ(μ,σ,a=0,b=1):
    α, β = (myclip_a - μ) / σ, (myclip_b - μ) / σ
    return (α,β)
    

## Necessary functions to run the ABC
- Model functions, which takes a dictionary of parameters and returns simulated observations of the same form as the observed data under some learning model. 
- Distance function 
- Parameter spaces
- Priors over the paramters (uniform)

In [19]:

αβ_lims = dict()
αβ_lims["LK"] = {"τ":(0,4), "p":(0,1)}
αβ_lims["LLK"] = {"τ":(0,4), "p":(0,1), "λ":(0,10)}
αβ_lims["PCHM"] = {"τ":(0,4), "p":(0,1)}
αβ_lims["LPCHM"] = {"τ":(0,4), "p":(0,1), "λ":(0,10)}
αβ_lims["JPCHM"] = {"τ":(0.,4.), "p":(0,1)}
αβ_lims["LJPCHM"] = {"τ":(0,4.), "p":(0,1), "λ":(0,10)}
αβ_lims["L1PCHM"] = {"τ":(0,4.), "p":(0,1), "λ":(0,10)}
αβ_lims["BR"] = {"ε":(0,1), "p":(0,1)}
αβ_lims["LBR"] = {"ε":(0,1), "p":(0,1), "λ":(0,10)}
αβ_lims["EWA"] = {"p":(0,1), "λ":(0,10), "φ":(0,1), "ρ":(0,1), "δ":(0,1)}


def LK_model(parameters):
    p = parameters["p"]
    τ = parameters["τ"]
    p_sd = parameters["p_sd"]
    τ_sd = parameters["τ_sd"]
    if "init_τ" in parameters: 
        init_params = np.array([parameters["init_τ"], parameters["init_λ"]])
    else:
         init_params=np.array([1.5,1.])
    hists = []
    for gid in gids:
        pop_LK = Population(games[gid][0], games[gid][1], rounds, p1_size,  p2_size, init_params=init_params, params_vec=[p,τ], σ_vec=[p_sd, τ_sd], lower_vec=[0.,0.], upper_vec=[1.,4.], random_params=True)
        hists.append(flatten_h(pop_LK.mul_runs_LK(n_runs)))
    flat_hists, shape = flatten_data(hists)
    return {"data": flat_hists, "shape": shape}


def LLK_model(parameters):
    p = parameters["p"]
    τ = parameters["τ"]
    λ = parameters["λ"]
    p_sd = parameters["p_sd"]
    τ_sd = parameters["τ_sd"] 
    λ_sd = parameters["λ_sd"]
    hists = []
    for gid in gids:
        pop_LLK = Population(games[gid][0], games[gid][1], rounds, p1_size,  p2_size, params_vec=[p,τ, λ], σ_vec=[p_sd, τ_sd, λ_sd], lower_vec=[0.,0., 0.], upper_vec=[1.,4., 10.], random_params=True)
        hists.append(flatten_h(pop_LLK.mul_runs_LLK(n_runs)))
    flat_hists, shape = flatten_data(hists)
    return {"data": flat_hists, "shape":shape}

def PCHM_model(parameters):
    p = parameters["p"]
    τ = parameters["τ"]
    p_sd = parameters["p_sd"]
    τ_sd = parameters["τ_sd"]
    if "init_τ" in parameters: 
        init_params = np.array([parameters["init_τ"], parameters["init_λ"]])
    else:
         init_params=np.array([1.5,1.])
    hists = []
    for gid in gids:
        pop_PCHM = Population(games[gid][0], games[gid][1], rounds, p1_size,  p2_size,  init_params=init_params, params_vec=[p,τ], σ_vec=[p_sd, τ_sd], lower_vec=[0.,0.], upper_vec=[1.,4.], random_params=True)
        hists.append(flatten_h(pop_PCHM.mul_runs_PCHM(n_runs)))
    flat_hists, shape = flatten_data(hists)
    return {"data": flat_hists, "shape": shape}

def JPCHM_model(parameters):
    p = parameters["p"]
    τ = parameters["τ"]
    p_sd = parameters["p_sd"]
    τ_sd = parameters["τ_sd"]
    if "init_τ" in parameters: 
        init_params = np.array([parameters["init_τ"], parameters["init_λ"]])
    else:
         init_params=np.array([1.5,1.])
    hists = []
    for gid in gids:
        pop_JPCHM = Population(games[gid][0], games[gid][1], rounds, p1_size,  p2_size,  init_params=init_params, params_vec=[p,τ], σ_vec=[p_sd, τ_sd], lower_vec=[0.,0.], upper_vec=[1.,αβ_lims["JPCHM"]["τ"][1]], random_params=True)
        hists.append(flatten_h(pop_JPCHM.mul_runs_JPCHM(n_runs)))
    flat_hists, shape = flatten_data(hists)
    return {"data": flat_hists, "shape": shape}


def LPCHM_model(parameters):
#     p = parameters["p"]
    τ = parameters["τ"]
    λ = parameters["λ"]
#     p_sd = parameters["p_sd"]
    τ_sd = parameters["τ_sd"]
    λ_sd = parameters["λ_sd"]
    if "init_τ" in parameters: 
        init_params = np.array([parameters["init_τ"], parameters["init_λ"]])
    else:
         init_params=np.array([1.5,1.])
    hists = []
    for gid in gids:
        pop_LPCHM = Population(games[gid][0], games[gid][1], rounds, p1_size,  p2_size,  init_params=init_params, params_vec=[τ, λ], σ_vec=[τ_sd, λ_sd], lower_vec=[0., 0.], upper_vec=[4., 10.], random_params=True)
        hists.append(flatten_h(pop_LPCHM.mul_runs_LPCHM(n_runs)))
    flat_hists, shape = flatten_data(hists)
    return {"data": flat_hists, "shape": shape}


def LJPCHM_model(parameters):
    p = parameters["p"]
    τ = parameters["τ"]
    λ = parameters["λ"]
    p_sd = parameters["p_sd"]
    τ_sd = parameters["τ_sd"]
    λ_sd = parameters["λ_sd"]
    if "init_τ" in parameters: 
        init_params = np.array([parameters["init_τ"], parameters["init_λ"]])
    else:
         init_params=np.array([1.5,1.])
    hists = []
    for gid in gids:
        pop_LJPCHM = Population(games[gid][0], games[gid][1], rounds, p1_size,  p2_size,  init_params=init_params, params_vec=[p, τ, λ], σ_vec=[p_sd, τ_sd, λ_sd], lower_vec=[0.,0., 0.], upper_vec=[1.,4., 10.], random_params=True)
        hists.append(flatten_h(pop_LJPCHM.mul_runs_LJPCHM(n_runs)))
    flat_hists, shape = flatten_data(hists)
    return {"data": flat_hists, "shape": shape}



def L1PCHM_model(parameters):
    p = parameters["p"]
    τ = parameters["τ"]
    λ = parameters["λ"]
    p_sd = parameters["p_sd"]
    τ_sd = parameters["τ_sd"]
    λ_sd = parameters["λ_sd"]
    if "init_τ" in parameters: 
        init_params = np.array([parameters["init_τ"], parameters["init_λ"]])
    else:
         init_params=np.array([1.5,1.])
    hists = []
    for gid in gids:
        pop_L1PCHM = Population(games[gid][0], games[gid][1], rounds, p1_size,  p2_size,  init_params=init_params, params_vec=[p, τ, λ], σ_vec=[p_sd, τ_sd, λ_sd], lower_vec=[0.,0., 0.], upper_vec=[1.,4., 10.], random_params=True)
        hists.append(flatten_h(pop_L1PCHM.mul_runs_L1PCHM(n_runs)))
    flat_hists, shape = flatten_data(hists)
    return {"data": flat_hists, "shape": shape}


def BR_model(parameters):
    p = parameters["p"]
    ε = parameters["ε"]
    p_sd = parameters["p_sd"]
    ε_sd = parameters["ε_sd"]
    if "init_τ" in parameters: 
        init_params = np.array([parameters["init_τ"], parameters["init_λ"]])
    else:
         init_params=np.array([1.5,1.])
    hists = []
    for gid in gids:
        pop_BR = Population(games[gid][0], games[gid][1], rounds, p1_size,  p2_size,  init_params=init_params, params_vec=[p,ε], σ_vec=[p_sd, ε_sd], lower_vec=[0.,0.], upper_vec=[1.,1.], random_params=True)
        hists.append(flatten_h(pop_BR.mul_runs_BR(n_runs)))
    flat_hists, shape = flatten_data(hists)
    return {"data": flat_hists, "shape": shape}


def LBR_model(parameters):
    p = parameters["p"]
#     ε = parameters["ε"]
    λ = parameters["λ"]
    p_sd = parameters["p_sd"]
#     ε_sd = parameters["ε_sd"]
    λ_sd = parameters["λ_sd"]
    if "init_τ" in parameters: 
        init_params = np.array([parameters["init_τ"], parameters["init_λ"]])
    else:
         init_params=np.array([1.5,1.])
    hists = []
    for gid in gids:
        pop_LBR = Population(games[gid][0], games[gid][1], rounds, p1_size,  p2_size,  init_params=init_params, params_vec=[p, λ], σ_vec=[p_sd, λ_sd], lower_vec=[0., 0.], upper_vec=[1., 10.], random_params=True)
        hists.append(flatten_h(pop_LBR.mul_runs_LBR(n_runs)))
    flat_hists, shape = flatten_data(hists)
    return {"data": flat_hists, "shape": shape}

def EWA_model(parameters):
    p = parameters["p"]
    p_sd = parameters["p_sd"]
    λ = parameters["λ"]
    λ_sd = parameters["λ_sd"]
    φ = parameters["φ"]
    φ_sd = parameters["φ_sd"]
    ρ = parameters["ρ"]
    ρ_sd = parameters["ρ_sd"]
    δ = parameters["δ"]
    δ_sd = parameters["δ_sd"]
    if "init_τ" in parameters: 
        init_params = np.array([parameters["init_τ"], parameters["init_λ"]])
    else:
         init_params=np.array([1.5,1.])
    hists = []
    for gid in gids:
        pop_EWA = Population(games[gid][0], games[gid][1], rounds, p1_size,  p2_size, init_params=init_params, params_vec=[p, λ, φ, ρ, δ], σ_vec=[p_sd, λ_sd, φ_sd, ρ_sd, δ_sd], lower_vec=[0.,0., 0., 0., 0.], upper_vec=[1.,10., 1., 1., 1.], random_params=True)
        hists.append(flatten_h(pop_EWA.mul_runs_EWA(n_runs)))
    flat_hists, shape = flatten_data(hists)
    return {"data": flat_hists, "shape": shape}

# def distance(x, y):
#     return 1 - calc_likelihood(x["data"], y["data"])/max_like

def distance(x,y):
    return max_like - calc_likelihood(x["data"], y["data"])

def euclidean_distance(x,y):
    simulated = unflatten_data(x["data"], x["shape"])
    hists = unflatten_data(y["data"], y["shape"]) 
    tot_distance = 0 
    for sim, hist in zip(simulated, hists):
        tot_distance += scp.spatial.distance.euclidean(sim,hist)
    return tot_distance
#     return scp.spatial.distance.euclidean(x["data"], y["data"])

# param_spaces = dict()
# param_spaces["LK"] = {"τ":(0.6, 3.), "τ_sd":(0,0.5), "p":(0., 1.), "p_sd":(0,0.5)}
# param_spaces["PCHM"] = {"τ":(0.6, 3.), "τ_sd":(0,0.5), "p":(0., 1.), "p_sd":(0,0.5)}
# param_spaces["BR"] = {"ε":(0,0.3), "ε_sd":(0,0.5), "p":(0.,1.), "p_sd":(0,0.5)}
# param_spaces["EWA"] = {"λ":(0,10), "λ_sd":(0,5), "p":(0., 1), "p_sd":(0,0.5), "φ":(0, 1), "φ_sd":(0,0.5), "ρ":(0,1), "ρ_sd":(0,0.5) , "δ":(0,1), "δ_sd":(0,0.5)}

# init_param_spaces = dict()
# init_param_spaces["LK"] = {"τ":(0.6, 3.), "τ_sd":(0,0.5), "p":(0., 1.), "p_sd":(0,0.5), "init_τ":(0,4), "init_λ":(0,10)}
# init_param_spaces["PCHM"] = {"τ":(0.6, 3.), "τ_sd":(0,0.5), "p":(0., 1.), "p_sd":(0,0.5), "init_τ":(0,4), "init_λ":(0,10)}
# init_param_spaces["BR"] = {"ε":(0,0.3), "ε_sd":(0,0.5), "p":(0.,1.), "p_sd":(0,0.5), "init_τ":(0,4), "init_λ":(0,10)}
# init_param_spaces["EWA"] = {"λ":(0,10), "λ_sd":(0,5), "p":(0., 1), "p_sd":(0,0.5), "φ":(0, 1), "φ_sd":(0,0.5), "ρ":(0,1), "ρ_sd":(0,0.5) , "δ":(0,1), "δ_sd":(0,0.5), "init_τ":(0,4), "init_λ":(0,10)}



# αβ_param_spaces = dict()
# αβ_param_spaces["LK"] = {"τ":(0., 100.), "τ_sd":(0,100.), "p":(0., 100.), "p_sd":(0,100.), "init_τ":(0,4), "init_λ":(0,10)}
# αβ_param_spaces["LLK"] = {"τ":(0., 100.), "τ_sd":(0,100.), "p":(0., 100.), "p_sd":(0,100.), "λ":(0,100.), "λ_sd":(0,100.), "init_τ":(0,4), "init_λ":(0,10)}
# αβ_param_spaces["PCHM"] = {"τ":(0., 100.), "τ_sd":(0,100.), "p":(0., 100.), "p_sd":(0,100.), "init_τ":(0,4), "init_λ":(0,10)}
# αβ_param_spaces["LPCHM"] = {"τ":(0., 100.), "τ_sd":(0,100.), "p":(0., 100.), "p_sd":(0,100.), "λ":(0,100.), "λ_sd":(0,100.), "init_τ":(0,4), "init_λ":(0,10)}
# αβ_param_spaces["BR"] = {"ε":(0,100.), "ε_sd":(0,100.), "p":(0.,100.), "p_sd":(0,100.), "init_τ":(0,4), "init_λ":(0,10)}
# αβ_param_spaces["EWA"] = {"λ":(0,10), "λ_sd":(0,100.), "p":(0., 100.), "p_sd":(0,100.), "φ":(0, 100.), "φ_sd":(0,100.), "ρ":(0, 100.), "ρ_sd":(0,100.) , "δ":(0,100.), "δ_sd":(0,100.), "init_τ":(0,4), "init_λ":(0,10)}


αβ_param_spaces = dict()
αβ_param_spaces["LK"] = {"τ":(0., 100.), "τ_sd":(0,100.), "p":(0., 100.), "p_sd":(0,100.), "init_τ":(0,4), "init_λ":(0,10)}
αβ_param_spaces["LLK"] = {"τ":(0., 100.), "τ_sd":(0,100.), "p":(0., 100.), "p_sd":(0,100.), "λ":(0,100.), "λ_sd":(0,100.), "init_τ":(0,4), "init_λ":(0,10)}
αβ_param_spaces["PCHM"] = {"τ":(0., 100.), "τ_sd":(0,100.), "p":(0., 100.), "p_sd":(0,100.), "init_τ":(0,4), "init_λ":(0,10)}
αβ_param_spaces["JPCHM"] = {"τ":(0., 100.), "τ_sd":(0,100.), "p":(0., 100.), "p_sd":(0,100.), "init_τ":(0,4), "init_λ":(0,10)}
αβ_param_spaces["LPCHM"] = {"τ":(0., 100.), "τ_sd":(0,100.), "p":(0., 100.), "p_sd":(0,100.), "λ":(0,100.), "λ_sd":(0,100.), "init_τ":(0,4), "init_λ":(0,10)}
αβ_param_spaces["LJPCHM"] = {"τ":(0., 100.), "τ_sd":(0,100.), "p":(0., 100.), "p_sd":(0,100.), "λ":(0,100.), "λ_sd":(0,100.), "init_τ":(0,4), "init_λ":(0,10)}
αβ_param_spaces["BR"] = {"ε":(0,100.), "ε_sd":(0,100.), "p":(0.,100.), "p_sd":(0,100.), "init_τ":(0,4), "init_λ":(0,10)}
αβ_param_spaces["LBR"] = {"ε":(0,100.), "ε_sd":(0,100.), "p":(0.,100.), "p_sd":(0,100.), "λ":(0,100.), "λ_sd":(0,100.),  "init_τ":(0,4), "init_λ":(0,10)}
αβ_param_spaces["EWA"] = {"λ":(0,10), "λ_sd":(0,100.), "p":(0., 100.), "p_sd":(0,100.), "φ":(0, 100.), "φ_sd":(0,100.), "ρ":(0, 100.), "ρ_sd":(0,100.) , "δ":(0,100.), "δ_sd":(0,100.), "init_τ":(0,4), "init_λ":(0,10)}


## With init_params sampling
# param_spaces = dict()
# param_spaces["LLK"] = {"τ":(0., 4.), "τ_sd":(0,0.1), "p":(0., 1.), "p_sd":(0,0.1), "λ":(0.,10.), "λ_sd":(0,1.), "init_τ":(0,4), "init_λ":(0,10)}
# param_spaces["LJPCHM"] = {"τ":(0., 4.), "τ_sd":(0,0.1), "p":(0., 1.), "p_sd":(0,0.1), "λ":(0.,10.), "λ_sd":(0,1.), "init_τ":(0,4), "init_λ":(0,10)}
# param_spaces["L1PCHM"] = {"τ":(0., 4.), "τ_sd":(0,0.1), "p":(0., 1.), "p_sd":(0,0.1), "λ":(0.,10.), "λ_sd":(0,1.), "init_τ":(0,4), "init_λ":(0,10)}
# param_spaces["LPCHM"] = {"τ":(0., 4.), "τ_sd":(0,0.1), "p":(0., 1.), "p_sd":(0,0.1), "λ":(0.,10.), "λ_sd":(0,1.), "init_τ":(0,4), "init_λ":(0,10)}
# param_spaces["LBR"] = {"p":(0.,1.), "p_sd":(0,0.1), "λ":(0.,10.), "λ_sd":(0,1.), "init_τ":(0,4), "init_λ":(0,10)}
# param_spaces["EWA"] = {"λ":(0,10), "λ_sd":(0,1.), "p":(0., 1), "p_sd":(0,0.1), "φ":(0, 1), "φ_sd":(0,0.1), "ρ":(0,1), "ρ_sd":(0,0.1) , "δ":(0,1), "δ_sd":(0,0.1), "init_τ":(0,4), "init_λ":(0,10)}




param_spaces = dict()
param_spaces["LLK"] = {"τ":(0., 4.), "τ_sd":(0,0.1), "p":(0., 1.), "p_sd":(0,0.1), "λ":(0.,10.), "λ_sd":(0,1.)}
param_spaces["LJPCHM"] = {"τ":(0., 4.), "τ_sd":(0,0.1), "p":(0., 1.), "p_sd":(0,0.1), "λ":(0.,10.), "λ_sd":(0,1.)}
param_spaces["L1PCHM"] = {"τ":(0., 3.), "τ_sd":(0,0.4), "p":(0., 1.), "p_sd":(0,0.2), "λ":(0.,10.), "λ_sd":(0,1.)}
param_spaces["LPCHM"] = {"τ":(0., 2.), "τ_sd":(0,0.3), "λ":(0.,10.), "λ_sd":(0,1.)}
param_spaces["LBR"] = {"p":(0.,1.), "p_sd":(0,0.1), "λ":(0.,10.), "λ_sd":(0,1.)}
param_spaces["EWA"] = {"λ":(0,10), "λ_sd":(0,1.), "p":(0., 1), "p_sd":(0,0.2), "φ":(0, 1), "φ_sd":(0,0.2), "ρ":(0,1), "ρ_sd":(0,0.2) , "δ":(0,1), "δ_sd":(0,0.2)}



αβ_params = dict()
αβ_params["LK"] = ["τ", "p"]
αβ_params["LLK"] = ["τ", "p", "λ"]
αβ_params["PCHM"] = ["τ", "p"]
αβ_params["JPCHM"] = ["τ", "p"]
αβ_params["LPCHM"] = ["τ", "p", "λ"]
αβ_params["LJPCHM"] = ["τ", "p", "λ"]
αβ_params["L1PCHM"] = ["τ", "p", "λ"]

αβ_params["BR"] = ["ε", "p"]
αβ_params["LBR"] = ["ε", "p", "λ"]
αβ_params["EWA"] = ["p", "λ", "φ", "ρ", "δ"]

 
αβ_plot_lims = copy.deepcopy(αβ_param_spaces)
for model in αβ_plot_lims:
    for param in αβ_params[model]:
        a,b = αβ_lims[model][param]
        αβ_plot_lims[model][param] = αβ_lims[model][param]
        αβ_plot_lims[model][param+"_sd"] = αβ_lims[model][param]
        
# model_names = ["LK", "LLK", "PCHM", "LPCHM", "JPCHM", "LJPCHM", "BR", "LBR", "EWA"]
# models = [LK_model, LLK_model, PCHM_model, LPCHM_model, JPCHM_model, LJPCHM_model, BR_model, LBR_model, EWA_model]

# model_names = ["LLK", "LPCHM", "LJPCHM", "LBR", "EWA"]
# models = [LLK_model, LPCHM_model, LJPCHM_model, LBR_model, EWA_model]

# model_names = ["LPCHM", "LJPCHM"]
# models = [LPCHM_model, LJPCHM_model]


# model_names = ["LLK", "LJPCHM", "LBR", "EWA"]
# models = [LLK_model, LJPCHM_model, LBR_model, EWA_model]

# model_names = ["L1PCHM", "LJPCHM", "LPCHM", "EWA"]
# models = [L1PCHM_model, LJPCHM_model, LPCHM_model, EWA_model]

model_names = ["LPCHM", "EWA"]
models = [LPCHM_model, EWA_model]

param_names = dict()
param_names["EWA"] = ["p", "λ", "φ", "ρ", "δ"]
param_names["L1PCHM"] = ["τ", "p", "λ"]
param_names["LPCHM"] = ["τ", "λ"]


# priors = [Distribution(**{key: RV("uniform", a, b - a)
#                         for key, (a,b) in param_spaces[mod].items()}) for mod in ["LK", "PCHM", "BR", "EWA"]]
# init_priors = [Distribution(**{key: RV("uniform", a, b - a)
#                         for key, (a,b) in init_param_spaces[mod].items()}) for mod in ["LK", "PCHM", "BR", "EWA"]]

# αβ_priors = [Distribution(**{key: RV("uniform", a, b - a)
#                         for key, (a,b) in αβ_param_spaces[mod].items()}) for mod in ["LK", "LLK", "PCHM", "LPCHM", "BR", "EWA"]]



priors = [Distribution(**{key: RV("uniform", a, b - a)
                        for key, (a,b) in param_spaces[mod].items()}) for mod in model_names]

# αβ_priors = [Distribution(**{key: RV("uniform", a, b - a)
#                         for key, (a,b) in αβ_param_spaces[mod].items()}) for mod in model_names]

In [ ]:
data_dict = data_TSE
gids = [1,2,3,4,5,6]
# gids = [4,5,6]
p1_size = 6
p2_size = 6 
rounds = 30
games = payoffs_dict[0]

n_runs = len(data_TSE[1])
# n_runs = 100
bw = 0.1
n_particles = 10000
max_pops = 30
min_accept_rate = 0.0001

## Running the abc-smc for the SSE pseudo_pops
Here we iterate over all the pseudo populations and run the abc-smc for each one and save down plots of the result. 

In [ ]:
for pseduo_pop in pseudo_data[gid]:
    flat_hists, shape = flatten_data([pseudo_data_correct[pseduo_pop][gid] for gid in gids]) 
    y = {"data": flat_hists}
    max_like = calc_likelihood(y["data"], y["data"])
    
    
    print(pseduo_pop + " - " + str(max_like) + " - " + str(n_particles)) 
#     abc = ABCSMC([LK_model, PCHM_model, BR_model, EWA_model], priors, distance, population_size=n_particles)
    abc = ABCSMC([LK_model, LLK_model, PCHM_model, LPCHM_model, BR_model, EWA_model], αβ_priors, distance, population_size=n_particles)
    db_path = ("sqlite:///" + os.path.join(tempfile.gettempdir(), "ABC.db"))
    print(db_path)
    abc.new(db_path, y, meta_info={"bw":bw, "Data":"Pseudo SSE", "Pseudo_pop":pseduo_pop, "Dist":"Beta"})
#     abc.new(db_path, y, meta_info={"bw":bw, "Data":"Pseudo SSE", "Pseudo_pop":pseduo_pop, "Dist":"Normal"})
    history = abc.run(minimum_epsilon=0.1, max_nr_populations=max_pops,  min_acceptance_rate=min_accept_rate)

    model_probabilities = history.get_model_probabilities()
    print(model_probabilities)
    model_probs = model_probabilities.plot.bar()
    fig = model_probs.get_figure()
    fig.savefig("fig/SSE/abc_modelprobs_beta" + str(n_particles) + "-" + str(max_pops) + "-bw"+ str(bw) + "-"+ pseduo_pop + ".png")

    # i = 0
    model_names = ["LK", "LLK", "PCHM", "LPCHM" "BR", "EWA"]
    for i in history.alive_models(history.max_t):
        df, w = history.get_distribution(m=i)
#         plot = plot_kde_matrix(df, w, limits=param_spaces[model_names[i]])
        plot = plot_kde_matrix(df, w, limits=αβ_param_spaces[model_names[i]])
        plot.savefig("fig/SSE/abc-beta" + str(n_particles) + "-" + str(max_pops)+ "-bw" + str(bw) + "-"  + pseduo_pop  + "-" + model_names[i] + ".png")
    plt.close("all")

In [ ]:
param_spaces

### Testing ground truth functionality


In [ ]:
pseudo_pop = "LK_1"
flat_hists, shape = flatten_data([pseudo_data_correct[pseduo_pop][gid] for gid in gids]) 
y = {"data": flat_hists}
max_like = calc_likelihood(y["data"], y["data"])

LK_1_params = {"p":0.9, "τ":0.9, "p_sd":0.1, "τ_sd":0.2}

print("Ground truth with: " + pseduo_pop + " - " + str(max_like) + " - " + str(n_particles)) 
abc = ABCSMC([LK_model, PCHM_model, BR_model, EWA_model], priors, distance, population_size=n_particles)
db_path = ("sqlite:///" + os.path.join(tempfile.gettempdir(), "ABC.db"))
print(db_path)
abc.new(db_path, y, gt_model=0, gt_par=LK_1_params)
history = abc.run(minimum_epsilon=0.1, max_nr_populations=max_pops)

model_probabilities = history.get_model_probabilities()
print(model_probabilities)
model_probs = model_probabilities.plot.bar()
fig = model_probs.get_figure()
fig.savefig("fig/abc_modelprobs" + str(n_particles) + "-" + str(max_pops) + "-bw"+ str(bw) + "-"+ pseduo_pop + ".png")

# i = 0
model_names = ["LK", "PCHM", "BR", "EWA"]
for i in history.alive_models(history.max_t):
    df, w = history.get_distribution(m=i)
    plot = plot_kde_matrix(df, w, limits=param_spaces[model_names[i]])
    plot.savefig("fig/abc-" + str(n_particles) + "-" + str(max_pops)+ "-bw" + str(bw) + "-"  + pseduo_pop  + "-" + model_names[i] + ".png")
plt.close("all")

# TSE data

### Loading the Data

In [ ]:
with open('data/data_dicts_TSE.pkl', 'rb') as input: 
    load_dict = pickle.load(input)
    actual_plays_dict = load_dict['actual_plays_dict']
    payoffs_dict = load_dict['payoffs_dict']
    
    
data_TSE = dict()

for gid in payoffs_dict[0]:
    payoffs_dict[0][gid] = [payoffs_dict[0][gid][i].astype(float) for i in range(2)]


for gid in games:
    data = [pop[gid] for pop in actual_plays_dict.values()]
    data_TSE[gid] = flatten_h(data)

### Setting the parameters

In [ ]:
data_dict = data_TSE
gids = [1,2,3,4,5,6]
# gids = [4,5,6]
p1_size = 6
p2_size = 6 
rounds = 30
games = payoffs_dict[0]

n_runs = len(data_TSE[1])
# n_runs = 100
bw = 0.1
n_particles = 10000
max_pops = 30
min_accept_rate = 0.0001

### Running the test for TSE data

In [ ]:
flat_hists, shape = flatten_data([data_dict[gid] for gid in gids]) 
y = {"data": flat_hists}
max_like = calc_likelihood(y["data"], y["data"])


print("TSE DATA: " + str(max_like) + " - " + str(n_particles)) 
# abc = ABCSMC([LK_model, PCHM_model, BR_model, EWA_model], priors, distance, population_size=n_particles)
abc = ABCSMC([LK_model, LLK_model, PCHM_model, LPCHM_model, BR_model, EWA_model], αβ_priors, distance, population_size=n_particles)
db_path = ("sqlite:///" + os.path.join(tempfile.gettempdir(), "ABC.db"))
print(db_path)
abc.new(db_path, y, meta_info={"bw":bw, "Data":"Real TSE - gids 456", "Distribution":"Beta"})
history = abc.run(minimum_epsilon=0.1, max_nr_populations=max_pops,  min_acceptance_rate=min_accept_rate)


model_probabilities = history.get_model_probabilities()
print(model_probabilities)
model_probs = model_probabilities.plot.bar()
fig = model_probs.get_figure()
fig.savefig("fig/TSE/TSE_modelprobs" + str(n_particles) + "-" + str(max_pops) + "-bw"+ str(bw) + "-"+ ".png")

# i = 0
model_names = ["LK", "LLK", "PCHM", "LPCHM", "BR", "EWA"]
for i in history.alive_models(history.max_t):
    df, w = history.get_distribution(m=i)
    plot = plot_kde_matrix(df, w, limits=αβ_param_spaces[model_names[i]])
    plot.savefig("fig/TSE/TSE-" + str(n_particles) + "-" + str(max_pops)+ "-bw" + str(bw) + "-"  + "-" + model_names[i] + ".png")
plt.close("all")

# MTurk actual data
### Loading data

In [20]:
with open('data_dicts.pkl', 'rb') as input: 
    load_dict = pickle.load(input)
    init_strats_dict = load_dict["init_strats_dict"]
    init_p1_dict = load_dict["init_p1_dict"]
    init_p2_dict = load_dict["init_p2_dict"]
    payoffs_dict = load_dict["payoffs_dict"]
    role_dict = load_dict["role_dict"]
    id_dict = load_dict["id_dict"]
    actual_plays_dict = load_dict["actual_plays_dict"]
    
for gid in range(1,4):
    payoffs_dict[gid][0] = payoffs_dict[gid][gid-1][0].astype(float)
    payoffs_dict[gid][1] = payoffs_dict[gid][gid-1][1].astype(float)

actuals_dict = dict()

for gid in actual_plays_dict:
    actuals_dict[gid] = flatten_single_hist(actual_plays_dict[gid])


### Setting parameters

In [21]:
data_dict = actuals_dict
# gids = [2,3,4,5,6]
gids = [1,2,3]
n_runs = 1
bw = 0.05
p1_size = 17
p2_size = 17 
rounds = 30
games = payoffs_dict
n_particles = 1000
max_pops = 10
min_accept_rate = 0.0001
init_ε = 10
α = 0.5


In [22]:
flat_hists, shape = flatten_data([np.array([data_dict[gid]]) for gid in gids]) 
shape

[(1, 180), (1, 300), (1, 210)]

In [27]:
y = {"data":flat_hists, "shape":shape}
max_like = calc_likelihood(y["data"], y["data"])

print(models)
print(model_names)

print("SSE DATA: " + str(max_like) + " - " + str(n_particles)) 
# abc = ABCSMC(models, αβ_priors, distance, population_size=n_particles)
# abc = ABCSMC(models, priors, euclidean_distance, population_size=n_particles,  eps=pyabc.epsilon.QuantileEpsilon(initial_epsilon=init_ε, alpha=α))
abc = ABCSMC(models, priors, distance, population_size=n_particles)

db_path = ("sqlite:///" + os.path.join(tempfile.gettempdir(), "ABC.db"))
abc.new(db_path, y, meta_info={"bw":bw, "Data":"Real MTurk", "distribution":"Truncated normal"})
# history = abc.run(minimum_epsilon=0.1, max_nr_populations=max_pops,  min_acceptance_rate=min_accept_rate)


[<function LPCHM_model at 0x7fe434dec1e0>, <function EWA_model at 0x7fe434dec488>]
['LPCHM', 'EWA']
SSE DATA: 1432.9876808410295 - 1000


Process Process-33:
Traceback (most recent call last):
  File "/home/gustav/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/gustav/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/gustav/anaconda3/lib/python3.6/site-packages/pyabc/sampler/multicore_evaluation_parallel.py", line 23, in work
    new_sim = simulate(new_param)
  File "/home/gustav/anaconda3/lib/python3.6/site-packages/pyabc/smc.py", line 385, in simulate_one
    par, self.summary_statistics)
  File "/home/gustav/anaconda3/lib/python3.6/site-packages/pyabc/model.py", line 97, in summary_statistics
    raw_data = self.sample(pars)
  File "/home/gustav/anaconda3/lib/python3.6/site-packages/pyabc/model.py", line 179, in sample
    return self.sample_function(pars)
  File "<ipython-input-19-f2a0d6a28e09>", line 92, in LPCHM_model
    pop_LPCHM = Population(games[gid][0], games[gid][1], rounds

Process Process-37:
IndexError: tuple index out of range
IndexError: tuple index out of range
Traceback (most recent call last):
  File "/home/gustav/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/gustav/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/gustav/anaconda3/lib/python3.6/site-packages/pyabc/sampler/multicore_evaluation_parallel.py", line 23, in work
    new_sim = simulate(new_param)
  File "/home/gustav/anaconda3/lib/python3.6/site-packages/pyabc/smc.py", line 385, in simulate_one
    par, self.summary_statistics)
  File "/home/gustav/anaconda3/lib/python3.6/site-packages/pyabc/model.py", line 97, in summary_statistics
    raw_data = self.sample(pars)
  File "/home/gustav/anaconda3/lib/python3.6/site-packages/pyabc/model.py", line 179, in sample
    return self.sample_function(pars)
Process Process-39:
  File "<ipython-input-19-f2a0d6a28

ProcessError: At least one worker is dead.

In [25]:
plt.switch_backend("Agg")

flat_hists, shape = flatten_data([np.array([data_dict[gid]]) for gid in gids]) 
y = {"data":flat_hists, "shape":shape}
max_like = calc_likelihood(y["data"], y["data"])

print(models)
print(model_names)

print("SSE DATA: " + str(max_like) + " - " + str(n_particles)) 
# abc = ABCSMC(models, αβ_priors, distance, population_size=n_particles)
# abc = ABCSMC(models, priors, euclidean_distance, population_size=n_particles,  eps=pyabc.epsilon.QuantileEpsilon(initial_epsilon=init_ε, alpha=α))
abc = ABCSMC(models, priors, distance, population_size=n_particles)

db_path = ("sqlite:///" + os.path.join(tempfile.gettempdir(), "ABC.db"))
abc.new(db_path, y, meta_info={"bw":bw, "Data":"Real MTurk", "distribution":"Truncated normal"})
history = abc.run(minimum_epsilon=0.1, max_nr_populations=max_pops,  min_acceptance_rate=min_accept_rate)



model_probabilities = history.get_model_probabilities()
print(model_probabilities)
model_probs = model_probabilities.plot.bar()
fig = model_probs.get_figure()
fig.savefig("fig/SSE/SSE_modelprobs" + str(n_particles) + "-" + str(max_pops) + "-bw"+ str(bw) + "-"+ ".png")

# i = 0


# for i in history.alive_models(history.max_t):
#     df, w = history.get_distribution(m=i)
#     df_copy = df.copy()
#     model = model_names[i]
# #     for param in αβ_params[model]:
# #         a, b = αβ_lims[model][param]
# #         df[param] = df_copy.apply(lambda x: st.beta.mean(x[param], x[param+"_sd"])*(b-a) + a, axis=1)
# #         df[param+"_sd"] = df_copy.apply(lambda x: st.beta.std(x[param], x[param+"_sd"])*(b-a), axis=1)
# #     plot = plot_kde_matrix(df, w, limits=αβ_plot_lims[model_names[i]])
#     plot = plot_kde_matrix(df[param_names[model]], w, limits=param_spaces[model_names[i]])
#     plot.savefig("fig/SSE/SSE-nostd-" + str(n_particles) + "-" + str(max_pops)+ "-bw" + str(bw) + "-"  + "-" + model_names[i] + ".png")
# plt.close("all")
# for i in history.alive_models(history.max_t):
#     df, w = history.get_distribution(m=i)
#     plot = plot_kde_matrix(df, w, limits=αβ_param_spaces[model_names[i]])
#     plot.savefig("fig/SSE/SSE-" + str(n_particles) + "-" + str(max_pops)+ "-bw" + str(bw) + "-"  + "-" + model_names[i] + ".png")
# plt.close("all")

[<function LPCHM_model at 0x7fe434dec1e0>, <function EWA_model at 0x7fe434dec488>]
['LPCHM', 'EWA']
SSE DATA: 1432.9876808410295 - 1000


Process Process-18:
Traceback (most recent call last):
  File "/home/gustav/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/gustav/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Process Process-17:
Traceback (most recent call last):
  File "/home/gustav/anaconda3/lib/python3.6/site-packages/pyabc/sampler/multicore_evaluation_parallel.py", line 23, in work
    new_sim = simulate(new_param)
  File "/home/gustav/anaconda3/lib/python3.6/site-packages/pyabc/smc.py", line 385, in simulate_one
    par, self.summary_statistics)
  File "/home/gustav/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/gustav/anaconda3/lib/python3.6/site-packages/pyabc/model.py", line 97, in summary_statistics
    raw_data = self.sample(pars)
  File "/home/gustav/anaconda3/lib/python3.6/site-packages/pyabc/model.py", line 179, in sample

Process Process-19:
  File "/home/gustav/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py", line 50, in _count_reduce_items
    items *= arr.shape[ax]
  File "/home/gustav/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
IndexError: tuple index out of range
Traceback (most recent call last):
  File "/home/gustav/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/gustav/anaconda3/lib/python3.6/site-packages/pyabc/sampler/multicore_evaluation_parallel.py", line 23, in work
    new_sim = simulate(new_param)
  File "/home/gustav/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/gustav/anaconda3/lib/python3.6/site-packages/pyabc/smc.py", line 385, in simulate_one
    par, self.summary_statistics)
  File "/home/gustav/anaconda3/lib/python3.6/site-packages/pyabc/sampler/multicore_eval

ProcessError: At least one worker is dead.

### Analyzing result

In [ ]:
model_probs = history.get_model_probabilities(history.max_t)["p"]
for i in history.alive_models(history.max_t):
    df, w = history.get_distribution(m=i)
    df_copy = df.copy()
    model = model_names[i]
    for param in αβ_params[model]:
        a, b = αβ_lims[model][param]
        df[param] = df_copy.apply(lambda x: st.beta.mean(x[param], x[param+"_sd"])*(b-a) + a, axis=1)
        df[param+"_sd"] = df_copy.apply(lambda x: st.beta.std(x[param], x[param+"_sd"])*(b-a), axis=1)
    avg_params = dict(df.multiply(w, axis=0).sum(0))
    print(model + " p=" + str(model_probs[i]) + " avg params:")
    print(avg_params)
    

In [ ]:
scp.spatial.distance.euclidean(y["data"], y["data"])

In [ ]:
df.multiply(w, axis=0).sum(0)

In [ ]:
history = abc.run(minimum_epsilon=0.1, max_nr_populations=next_pops,  min_acceptance_rate=min_accept_rate)

### Plotting and comparing acutal to simulated

In [ ]:
rgb_cols = [(228/255, 26/255, 28/255), (55/255, 126/255, 184/255), (77/255, 175/255, 74/255), (152/255, 78/255, 163/255), (255/255, 127/255, 0/255)]

def plot_h(h, save=False):
    plt.figure(figsize=(24,8))
    for role in range(2):
        ax = plt.subplot(1,2,(role + 1))
        ax.set_ylim([-0.01,1.01])
        ax.set_yticks([0,0.25,0.5,0.75,1])
        n_s = h[role].shape[1]
        plt.title("Player role " + str(role + 1))
        for s in range(n_s):
            plt.plot(h[role][:,s], color=rgb_cols[s], ls="-", label="Strat "+str(s))
        ax.legend()
    if save:
        plt.savefig(save)
        plt.close()
    else:
        plt.show()

In [ ]:
plt.switch_backend("module://ipykernel.pylab.backend_inline")
i = 0
sess = 0
# models = [LK_model, LLK_model, PCHM_model, LPCHM_model, JPCHM_model, BR_model, EWA_model]

# df, w = history.get_distribution(m=i)



In [ ]:
i = 0
params = dict(df.sample(axis=0, weights=w).iloc[0])
ε_sim = 2000000 
ε_min = ε_sim
gids = [2,4,5,6]

flat_hists, shape = flatten_data([np.array([data_dict[gid]]) for gid in gids]) 
y = {"data": flat_hists, "shape":shape}

test_params = dict()
test_params["LPCHM"] = {"τ": 1.24, "τ_sd":0.1, "p":0.25, "p_sd":0.1, "λ":4.23, "λ_sd":0.3}
test_params["EWA"] = {'λ': 3.7, 'λ_sd': 0.2, 'p': 1., 'p_sd': 0.1, 'φ': 0.39, 'φ_sd': 0.1, 'ρ': 0.21, 'ρ_sd': 0.1, 'δ': 0.76, 'δ_sd': 0.1}


iters = 0
iters_lim = 10000
while (ε_sim > 5) & (iters < iters_lim):
    iters += 1
    if iters % 200 == 0:
        print("Iter: " + str(iters))
    params = dict(df.sample(axis=0, weights=w).iloc[0])
#     params = test_params[model_names[i]]
    sim = models[i](params)
    ε_sim = euclidean_distance(sim, y)
    if ε_min > ε_sim:
        ε_min = ε_sim
        print("ε_min: " + str(ε_min))
    


# μσ_params = copy.deepcopy(params)
# model = model_names[i]
# for param in αβ_params[model]:
#     a, b = αβ_lims[model][param]
#     μσ_params[param] = st.beta.mean(params[param], params[param+"_sd"])*(b-a) + a
#     μσ_params[param+"_sd"] = st.beta.std(params[param], params[param+"_sd"])*(b-a)

# print(μσ_params)

# sim = models[i](params)
sim_hists = sim["data"]
shape = sim["shape"]
sim_plays_flat = unflatten_data(sim_hists, shape)

print(distance(sim, y))


list_gid = 0
for gid in gids:
    
    print("------- " + str(gid) + "------") 
    p1_s, p2_s = games[gid][0].shape

    hist = sim_plays_flat[list_gid]
    sim_play = [hist[0:2][sess][0:p1_s*rounds].reshape((rounds,p1_s)), hist[0:2][sess][p1_s*rounds:].reshape((rounds,p2_s))]
    actual_play = actual_plays_dict[gid]

    print("ACTUAL")
    plot_h(actual_play)
    print("Simulated")
    if gid == 2:
        plot_h(sim_play, save="close_gid2_png")
    plot_h(sim_play)
    list_gid += 1



In [ ]:

list_gid = 0
for gid in gids:
    
    print("------- " + str(gid) + "------") 
    p1_s, p2_s = games[gid][0].shape

    hist = sim_plays_flat[list_gid]
    sim_play = [hist[0:2][sess][0:p1_s*rounds].reshape((rounds,p1_s)), hist[0:2][sess][p1_s*rounds:].reshape((rounds,p2_s))]
    actual_play = actual_plays_dict[gid]

    print("ACTUAL")
    plot_h(actual_play)
    print("Simulated")
    if gid == 2:
        plot_h(sim_play, save="close_gid2_png")
    plot_h(sim_play)
    list_gid += 1

In [ ]:
sim["data"]

### Plotting evolution of params-estimates

In [ ]:
# for i in history.alive_models(history.max_t):
i = 0
model_name = model_names[i]
for param in param_names[model_name]:
    fig, ax = plt.subplots()
    for t in range(history.max_t+1):
        df, w = history.get_distribution(m=i, t=t)
    #                 df_copy = df.copy()
    #                 a, b = αβ_lims[model_name][param]
    #                 df[param] = df_copy.apply(lambda x: st.beta.mean(x[param], x[param+"_sd"])*(b-a) + a, axis=1)
    # #                 df[param+"_sd"] = df_copy.apply(lambda x: st.beta.std(x[param], x[param+"_sd"])*(b-a), axis=1)
        plot_kde_1d(df, w,
                    xmin=param_spaces[model_name][param][0], xmax=param_spaces[model_name][param][1],
                    x=param, ax=ax,
                    label="PDF t={}".format(t))
#         ax.axvline(observation, color="k", linestyle="dashed");
    ax.legend();
    plt.savefig("fig/params/actual-" + param + "-" + str(n_particles) + "-" + str(max_pops) + "-" + model_names[i] +".png")
    plt.show()

In [ ]:
parameters = priors[0].rvs()
parameters = αβ_priors[0].rvs()
gid = 2
print(parameters)


p = parameters["p"]
τ = parameters["τ"]
p_sd = parameters["p_sd"]
τ_sd = parameters["τ_sd"]
if "init_τ" in parameters: 
    init_params = np.array([parameters["init_τ"], parameters["init_λ"]])
else:
     init_params=np.array([1.5,1.])
pop_LK = Population(games[gid][0], games[gid][1], rounds, p1_size,  p2_size, init_params=init_params, params_vec=[p,τ], σ_vec=[p_sd, τ_sd], lower_vec=[0.,0.], upper_vec=[1,3], random_params=True)

print(pop_LK.p1_params)
print(pop_LK.p2_params)

%timeit pop_LK.mul_runs_LK(10)

In [ ]:
pop_LK = Population(games[2][0], games[2][1], rounds, p1_size,  p2_size, params_vec=[2.,2.], σ_vec=[2., 2.], lower_vec=[0.,0.], upper_vec=[1.,20.], random_params=True)

In [ ]:
@jit(nopython=True)
def draw_beta(μ_in, σ_in, a, b):
    μ = (μ_in-a)/(b-a)
    σ = σ_in/(b-a)
    if σ**2 > (1-μ)*μ:
        pass
        print("σ is to big", σ, (1-μ)*μ)
    σ = min(σ, np.sqrt((1-μ)*μ) - 0.001)
    α = ((1-μ)/σ**2 - 1/μ)*μ**2
    β = α*(1/μ - 1)
#     print(α,β)
    return np.random.beta(α,β)*(b-a) + a

In [ ]:
for i in history.alive_models(history.max_t):
    df, w = history.get_distribution(m=i)
    df_copy = df.copy()
    model = model_names[i]
    for param in αβ_params[model]:
        a, b = αβ_lims[model][param]
        df[param] = df_copy.apply(lambda x: st.beta.mean(x[param], x[param+"_sd"])*(b-a) + a, axis=1)
        df[param+"_sd"] = df_copy.apply(lambda x: st.beta.std(x[param], x[param+"_sd"])*(b-a), axis=1)


# df.apply(lambda x: print(x["p"]/(x["p"] + x["p_sd"])), axis=1)

In [ ]:
model_names = ["LK", "LLK", "PCHM", "LPCHM", "BR", "EWA"]
for i in history.alive_models(history.max_t):
    df, w = history.get_distribution(m=i)
    df_copy = df.copy()
    model = model_names[i]
    for param in αβ_params[model]:
        a, b = αβ_lims[model][param]
        df[param] = df_copy.apply(lambda x: st.beta.mean(x[param], x[param+"_sd"])*(b-a) + a, axis=1)
        df[param+"_sd"] = df_copy.apply(lambda x: st.beta.std(x[param], x[param+"_sd"])*(b-a), axis=1)
    plot = plot_kde_matrix(df, w, limits=αβ_plot_lims[model_names[i]])
    plot.savefig("fig/SSE/SSE-" + str(n_particles) + "-" + str(max_pops)+ "-bw" + str(bw) + "-"  + "-" + model_names[i] + ".png")
plt.close("all")

In [ ]:
αβ_priors[1].rvs()

In [ ]:
αβ_lims["JPCHM"]["τ"][1]